# prepare

In [14]:
# imports
import os
import argparse
import json

import torch
import pytorch_lightning as pl
import torchmetrics
import transformers

from utils import (
    PersonaDataset,
    GenerativeCollator,
    RetrievalCollator,
    aggregate_encoder_output,
    sim_func,
)
from models import RetrievalModel, GenerativeModel

In [15]:
# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

In [16]:
# config bert
parser = argparse.ArgumentParser()
bert_args = parser.parse_args("")
with open("configs/bert_config.json", "r") as config:
    opt = json.load(config)
vars(bert_args).update(opt)

# pretrained model

In [20]:
# bert tokenizer
with open(bert_args.special_tokens_dict, "r") as config:
    special_tokens_dict = json.load(config)

bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    bert_args.pretrained_bert,
    truncation_side=bert_args.truncation_side,
    padding_side=bert_args.padding_side,
)
bert_tokenizer.add_special_tokens(special_tokens_dict)

7

In [5]:
# bert
context_bert = transformers.AutoModel.from_pretrained(bert_args.pretrained_bert)
context_bert.resize_token_embeddings(len(bert_tokenizer))
candidate_bert = transformers.AutoModel.from_pretrained(bert_args.pretrained_bert)
candidate_bert.resize_token_embeddings(len(bert_tokenizer))

Some weights of the model checkpoint at pretrain_models/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at pretrain_models/rubert-base-cased-conversational wer

Embedding(100799, 768)

# data

In [6]:
# dataset
dataset = PersonaDataset(bert_args.data_path, mod='get_examples_gk')
train_size = len(dataset) - len(dataset) // bert_args.val_split
val_size = len(dataset) // bert_args.val_split
vars(bert_args).update({"train_size": train_size, "val_size": val_size})
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

In [21]:
# bert_callator
bert_callator = RetrievalCollator(
    bert_tokenizer, padding=bert_args.padding, max_length=bert_args.context_len
)

Using eos_token, but it is not set yet.


In [8]:
# dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=bert_args.batch_size,
    shuffle=True,
    collate_fn=bert_callator,
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=bert_args.batch_size,
    shuffle=False,
    collate_fn=bert_callator,
)

In [9]:
# scheduler len
scheduler_len = len(train_dataloader) * bert_args.epochs

# pl trainloop

In [10]:
# pl model
model = RetrievalModel(
    context_bert,
    candidate_bert,
    bert_args.batch_size,
    scheduler_len,
    bert_args.num_warmup_steps,
    bert_args.lr,
    aggregation_mod=bert_args.aggregation_mod,
    sim_mod=bert_args.sim_mod
)

/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'context_BERT' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['context_BERT'])`.
  rank_zero_warn(
/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'candidat_BERT' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['candidat_BERT'])`.
  rank_zero_warn(


In [11]:
# logger
logger = pl.loggers.comet.CometLogger(
    api_key=bert_args.api_key,
    save_dir=bert_args.save_dir,
    project_name=bert_args.project_name,
    experiment_name=bert_args.experiment_name,
)
logger.log_hyperparams(bert_args)

CometLogger will be initialized in online mode
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/anpopaicoconat/bi-encoder/87fefd24f84f4ea3b93b389f01932598



In [12]:
# trainer
trainer = pl.Trainer(
    max_epochs=bert_args.epochs,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=bert_args.gradient_clip_val,
    logger=logger,
    num_sanity_val_steps=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
# fit
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | context_BERT  | BertModel        | 163 M 
1 | candidat_BERT | BertModel        | 163 M 
2 | loss          | CrossEntropyLoss | 0     
3 | train_metrics | MetricCollection | 0     
4 | val_metrics   | MetricCollection | 0     
---------------------------------------------------
326 M     Trainable params
0         Non-trainable params
326 M     Total params
1,307.640 Total estimated model params size (MB)
COMET WARNING: truncated string; too long: 'BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(100799, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAt

Sanity Checking: 0it [00:00, ?it/s]

/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 6:  32%|███▏      | 299/949 [03:47<08:14,  1.31it/s, loss=0.891, v_num=2598, lr=4.14e-5, train_loss=0.857, val_loss_step=2.150, val_loss_epoch=2.980]

/home/stc/rybin-as/miniconda3/envs/persona/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
